## 라이브러리 임포트

In [ ]:
import re

## 데이터 추출

In [ ]:
# 추출 경로
data_path=r'C:\Users\안성규\Desktop\법률 LLM 프로젝트\data\R078r3e 파일럿 데이터.txt'

# 텍스트 데이터 추출
with open(data_path, 'rb') as source:
    lines = source.readlines()
    lines = [element.decode('utf-8') for element in lines]

text_sum = ''.join(lines)
print(text_sum)

## 데이터 가공

In [ ]:
# 주석 제거
 ## 주석 시작 행 확인
pattern = r'\s[1-9]\s'
idxl_start = [index for index, line in enumerate(lines) if re.match(pattern, line)]

 ## 주석 종료 행 확인 및 범위 특정
idxl_remove = []
for idx_start in idxl_start:
    idx_moment = idx_start

    while("[END OF PAGE]" not in lines[idx_moment]):
        idxl_remove.append(idx_moment)
        idx_moment = idx_moment + 1

 ## 주석 제거
lines_processed = lines.copy()
for idx_remove in sorted(idxl_remove, reverse=True):
    lines_processed.pop(idx_remove)

# 특수 데이터 제거
 ## 특수 데이터 시작 시점 확인
pattern_colon = r".*\:\s{2}\r\n$"
idxL_start = [index + 1 for index, line in enumerate(lines_processed) if re.match(pattern_colon, line)]


pattern_article = r'^\s*\d{1,}\.\d{1,}\.'
pattern_item = r"^\(.*\)"

 ## 특수 데이터 종료 시점 확인 및 데이터 범위 특정
idxL_remove = []
for idx_start in idxL_start:

     ### 시작 지점부터 한 줄씩 종료 조건을 확인하며 데이터 범위 탐색
    idx_moment = idx_start
    #print(f"Start with the line IDX:{idx_moment}")

    while not (re.match(pattern_article, lines_processed[idx_moment]) or re.match(pattern_item, lines_processed[idx_moment])):
        idxL_remove.append(idx_moment)
        #print(f" Mark the line IDX to be removed:{idx_moment}")

        idx_moment = idx_moment + 1

     ### 포함 여부 확인 후 작업 스케줄 수정 (특수 데이터의 범위로 특정된 부분 ~ 기존에 확인한 탐색 시작 지점)
    idxL_redundant = [ idxL_start.index(elm_redundant) for elm_redundant in list(filter(lambda x : (idx_start < x <= idx_moment), idxL_start)) ]
    for idx_redundant in sorted(idxL_redundant, reverse=True):
        idxL_start.pop(idx_redundant)
         #print(f"Pop out the start IDX of the redundant: {idxL_start.pop(idx_redundant)}")

 ## 특정한 범위를 기준으로 특수 데이터 제거
for idx_remove in sorted(idxL_remove, reverse=True):
    lines_processed.pop(idx_remove)

# 띄어쓰기 제거 (페이지 전환 부분)

 ## 페턴
pattern_withSpace = r'^\s+'
pattern_endPage = r'^\[END OF PAGE\]'

 ## 페이지 종료 시점 확인
idxL_pageEnd = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]
idxL_pageEnd.pop()

 ## 페이지 종료 후 띄어쓰기 삽입 여부 확인 및 수정
for idx_pageEnd in idxL_pageEnd:
    if re.match(pattern_withSpace, lines_processed[idx_pageEnd+1]):
        lines_processed[idx_pageEnd+1] = lines_processed[idx_pageEnd+1][1:]

# 띄어쓰기 제거 (예외 처리)

pattern_withSpace_exceptions = r'^\s\d{1,}\.\d{1,}\.'
idxL_withSpace_exceptions = [index for index, line in enumerate(lines_processed) if re.match(pattern_withSpace_exceptions, line)]

for idx_withSpace_exceptions in idxL_withSpace_exceptions:
    lines_processed[idx_withSpace_exceptions] = lines_processed[idx_withSpace_exceptions][1:]

# 단독 개행 및 페이지 전환 제거

## 패턴
pattern_onlyEnter = r'^\s*\r\n$'
pattern_endPage = r'^\[END OF PAGE\]'

## 패턴과 일치하는 행 확인
idxL_onlyEnter_endPage = [index for index, line in enumerate(lines_processed) if (re.match(pattern_onlyEnter, line) or re.match(pattern_endPage, line))]

## 패턴과 일치하는 행 제거
for idx_remove in sorted(idxL_onlyEnter_endPage, reverse=True):
    lines_processed.pop(idx_remove)

# 챕터 제목 행 병합
pattern_chapterTitle = r'^\s{2}[a-zA-Z]'
idxL_chapterTitle = [index for index, line in enumerate(lines_processed) if (re.match(pattern_chapterTitle, line))]

for idx_chapterTitle in sorted(idxL_chapterTitle, reverse=True):
    lines_processed[idx_chapterTitle-1] = lines_processed[idx_chapterTitle-1][:-4]

# 조문 행 병합
pattern_normalText = r'^[a-zA-Z]'
idxL_normalText = [index for index, line in enumerate(lines_processed) if (re.match(pattern_normalText, line))]

for idx_normalText in sorted(idxL_normalText, reverse=True):
    lines_processed[idx_normalText-1] = lines_processed[idx_normalText-1][:-2]

# 조문 행 병합 (예외 처리)
pattern_normalText_exceptions = r'^\(.{4,}\)'
idxL_normalText_exceptions = [index for index, line in enumerate(lines_processed) if (re.match(pattern_normalText_exceptions, line))]

for idx_normalText_exceptions in sorted(idxL_normalText_exceptions, reverse=True):
    lines_processed[idx_normalText_exceptions-1] = lines_processed[idx_normalText_exceptions-1][:-2]

## 데이터 저장

In [ ]:
# 결과 저장 경로
data_path=r'C:\Users\안성규\Desktop\법률 LLM 프로젝트\data\R078r3e_processed.txt'

# txt 파일 저장
with open(data_path, 'w', encoding='utf-8') as result:
    text_sum = ''.join(lines_processed)
    result.write(text_sum)